In [1]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
#from costcla.models import CostSensitiveDecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

C:\Users\DELL\Anaconda3\lib\site-packages\dask\dataframe\utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('D:\\Data Storm 2.0\\Hotel-A-train.csv')

In [4]:
df.columns

Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type', 'Expected_checkin',
       'Expected_checkout', 'Booking_date',
       'time between booking date and check in date', 'time of stay', 'Adults',
       'Children', 'Babies', 'Meal_Type', 'Visted_Previously',
       'Previous_Cancellations', 'Deposit_type', 'Booking_channel',
       'Required_Car_Parking', 'Reservation_Status', 'Use_Promotion',
       'Discount_Rate', 'Room_Rate', 'Final Rate'],
      dtype='object')

In [22]:
#label encoding categorical variables
obj_df = new_df.select_dtypes(include=['object']).copy()
obj_df.head()

,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion
0,Latino,Grad,<25K,North,City Hotel,BB,No,No,No Deposit,Online,Yes,Check-In,Yes
1,Latino,Mid-School,50K -- 100K,East,City Hotel,BB,No,No,Refundable,Online,Yes,Check-In,No
2,caucasian,Grad,<25K,East,City Hotel,BB,No,No,No Deposit,Online,Yes,Check-In,No
3,African American,College,>100K,South,Airport Hotels,BB,No,No,Refundable,Agent,Yes,Check-In,Yes
4,Latino,High-School,25K --50K,East,Resort,BB,No,No,No Deposit,Direct,No,Check-In,Yes


In [23]:
obj_df.columns

Index(['Ethnicity', 'Educational_Level', 'Income', 'Country_region',
       'Hotel_Type', 'Meal_Type', 'Visted_Previously',
       'Previous_Cancellations', 'Deposit_type', 'Booking_channel',
       'Required_Car_Parking', 'Reservation_Status', 'Use_Promotion'],
      dtype='object')

In [3]:
label_encoder = LabelEncoder()

In [5]:
new_df['Ethnicity']= label_encoder.fit_transform(new_df['Ethnicity'])
new_df['Educational_Level']= label_encoder.fit_transform(new_df['Educational_Level'])
new_df['Income']= label_encoder.fit_transform(new_df['Income'])                                                        
new_df['Country_region']= label_encoder.fit_transform(new_df['Country_region'])
new_df['Hotel_Type']= label_encoder.fit_transform(new_df['Hotel_Type'])
new_df['Meal_Type']= label_encoder.fit_transform(new_df['Meal_Type'])
#new_df['Visted_Previously']= label_encoder.fit_transform(new_df['Visted_Previously'])
#new_df['Previous_Cancellations']= label_encoder.fit_transform(new_df[ 'Previous_Cancellations'])
new_df['Deposit_type']= label_encoder.fit_transform(new_df['Deposit_type'])
new_df['Booking_channel']= label_encoder.fit_transform(new_df['Booking_channel'])
new_df['Required_Car_Parking']= label_encoder.fit_transform(new_df['Required_Car_Parking'])
new_df['Reservation_Status']= label_encoder.fit_transform(new_df['Reservation_Status'])
#new_df['Use_Promotion']= label_encoder.fit_transform(new_df['Use_Promotion'])

In [4]:
#Drop unnecessary variables
new_df = df.drop(['Reservation-id','Gender','Expected_checkin','Expected_checkout','Booking_date','Final Rate','Previous_Cancellations','Use_Promotion','Visted_Previously'],
                 axis=1)

In [65]:
x_train = new_df.drop(['Reservation_Status'], axis=1)

In [66]:
y_train = new_df['Reservation_Status']
y_train = y_train+1

In [64]:
y_train.unique

<bound method Series.unique of 0        2
1        2
2        2
3        2
4        2
        ..
27494    2
27495    3
27496    2
27497    1
27498    3
Name: Reservation_Status, Length: 27499, dtype: int32>

In [8]:
val = pd.read_csv('D:\\Data Storm 2.0\\Hotel-A-validation.csv')
val.columns

Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type', 'Expected_checkin',
       'Expected_checkout', 'Booking_date',
       'time between booking date and check in date', 'time of stay', 'Adults',
       'Children', 'Babies', 'Meal_Type', 'Visted_Previously',
       'Previous_Cancellations', 'Deposit_type', 'Booking_channel',
       'Required_Car_Parking', 'Reservation_Status', 'Use_Promotion',
       'Discount_Rate', 'Room_Rate'],
      dtype='object')

In [9]:
#label encoding the variables in the validation set
val['Ethnicity']= label_encoder.fit_transform(val['Ethnicity'])
val['Educational_Level']= label_encoder.fit_transform(val['Educational_Level'])
val['Income']= label_encoder.fit_transform(val['Income'])                                                        
val['Country_region']= label_encoder.fit_transform(val['Country_region'])
val['Hotel_Type']= label_encoder.fit_transform(val['Hotel_Type'])
val['Meal_Type']= label_encoder.fit_transform(val['Meal_Type'])
val['Visted_Previously']= label_encoder.fit_transform(val['Visted_Previously'])
val['Previous_Cancellations']= label_encoder.fit_transform(val[ 'Previous_Cancellations'])
val['Deposit_type']= label_encoder.fit_transform(val['Deposit_type'])
val['Booking_channel']= label_encoder.fit_transform(val['Booking_channel'])
val['Required_Car_Parking']= label_encoder.fit_transform(val['Required_Car_Parking'])
val['Reservation_Status']= label_encoder.fit_transform(val['Reservation_Status'])
val['Use_Promotion']= label_encoder.fit_transform(val['Use_Promotion'])

In [67]:
x_val = val.drop(['Reservation-id','Gender','Expected_checkin','Expected_checkout','Booking_date','Previous_Cancellations','Use_Promotion','Visted_Previously',
                 'Reservation_Status'], axis=1)
y_val = val['Reservation_Status']

In [11]:
y_val = y_val+1

In [12]:
y_val.head()

0    3
1    1
2    1
3    2
4    2
Name: Reservation_Status, dtype: int32

DECISION TREE CLASSIFIER

In [68]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)
y_pred_tree = clf.predict(x_val)

In [69]:
dt_f1_score = f1_score(y_val, y_pred_tree, average='weighted')
dt_f1_score

0.1979234292050526

In [19]:
### Fitting XGBoost Model
xgb = XGBClassifier(learning_rate=0.3775, max_depth=5, n_estimators=100, random_state=2)
xgb.fit(x_train,y_train)

## Making predctions
xgb_predict = xgb.predict(x_val)


In [20]:
## F1 Score
xgb_fscore = f1_score(y_val,xgb_predict,average='weighted')
print(xgb_fscore)

0.43733684143945756


In [15]:
#### Parameter tuning
params_grid = {
    'max_depth':[1,2,3,4,5,6],
    'n_estimators':[5,10,25,50,100],
    'learning_rate': np.linspace(0.01,0.5,num=8,endpoint=False)
}

In [16]:
params_fixd = {
    'objective': 'multi:softmax',
}

In [17]:
best_grid = GridSearchCV(
    estimator=XGBClassifier(**params_fixd,random_state=2),
#    estimator=GradientBoostingClassifier(random_state=2),
#    estimator=AdaBoostClassifier(random_state=2),
    param_grid = params_grid,
    scoring='f1_weighted'
)

In [18]:
## Parameter tuning for extreme gradient boosting XGBoost
best_grid.fit(x_train,y_train)
print(best_grid.best_score_,best_grid.best_params_)

0.6738298066349822 {'learning_rate': 0.3775, 'max_depth': 5, 'n_estimators': 100}


In [33]:
#loading the test set
test = pd.read_csv('D:\\Data Storm 2.0\\Hotel-A-test.csv')

In [34]:
test['Ethnicity']= label_encoder.fit_transform(test['Ethnicity'])
test['Educational_Level']= label_encoder.fit_transform(test['Educational_Level'])
test['Income']= label_encoder.fit_transform(test['Income'])                                                        
test['Country_region']= label_encoder.fit_transform(test['Country_region'])
test['Hotel_Type']= label_encoder.fit_transform(test['Hotel_Type'])
test['Meal_Type']= label_encoder.fit_transform(test['Meal_Type'])
test['Visted_Previously']= label_encoder.fit_transform(test['Visted_Previously'])
test['Previous_Cancellations']= label_encoder.fit_transform(test[ 'Previous_Cancellations'])
test['Deposit_type']= label_encoder.fit_transform(test['Deposit_type'])
test['Booking_channel']= label_encoder.fit_transform(test['Booking_channel'])
test['Required_Car_Parking']= label_encoder.fit_transform(test['Required_Car_Parking'])
#test['Reservation_Status']= label_encoder.fit_transform(test['Reservation_Status'])
test['Use_Promotion']= label_encoder.fit_transform(test['Use_Promotion'])

In [35]:
x_test = test.drop(['Reservation-id','Gender','Expected_checkin','Expected_checkout','Booking_date','Use_Promotion','Previous_Cancellations','Visted_Previously'],axis=1)

In [36]:
y_test = xgb.predict(x_test)

In [48]:
test_df.tail()

,Reservation_Status,Reservation-id
4313,2,39110574
4314,2,5496554
4315,2,59004046
4316,2,65838682
4317,1,14474101


In [37]:
test_df = pd.DataFrame(y_test, columns = ['Reservation_Status'])

In [38]:
test_df['Reservation-id'] = test['Reservation-id']

In [39]:
column_titles = ['Reservation-id','Reservation_Status']
test_results1 = test_df.reindex(columns=column_titles)

In [40]:
test_results1.head()

,Reservation-id,Reservation_Status
0,62931593,2
1,70586099,2
2,4230648,2
3,25192322,2
4,80931528,2


In [41]:
test_results1.to_csv('submission_2.csv')

DAY 2 - NAIVE BAYES ALGORITHM

In [14]:
gnb = GaussianNB()

In [15]:
gnb.fit(x_train, y_train)

GaussianNB()

In [16]:
y_pred_NB = gnb.predict(x_val)

In [17]:
nb_fscore = f1_score(y_val, y_pred_NB, average='weighted')

In [18]:
nb_fscore

0.43263349387055267

#hyper parameter tuning for decision tree classifier

In [23]:
param_grid = {
    "criterion":['gini','entropy'],
    "max_depth":range(1,10),
    "min_samples_split":range(1,10),
    "min_samples_leaf":range(1,5)
    
}

In [24]:
grid_search = GridSearchCV(clf,
                          cv=10,
                          param_grid=param_grid,
                          verbose=1,
                          n_jobs=-1,
                          scoring='f1_weighted')

In [26]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 648 candidates, totalling 6480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 443 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 1443 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 2843 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 4643 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 6480 out of 6480 | elapsed:  1.9min finished


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 10),
                         'min_samples_leaf': range(1, 5),
                         'min_samples_split': range(1, 10)},
             scoring='f1_weighted', verbose=1)

In [29]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 7,
 'min_samples_leaf': 2,
 'min_samples_split': 4}

In [55]:
dt2 = tree.DecisionTreeClassifier(max_depth=7, min_samples_leaf=2, min_samples_split=4)
dt2.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=7, min_samples_leaf=2, min_samples_split=4)

In [56]:
y_new_dt = dt2.predict(x_val)
print(f1_score(y_val,y_new_dt,average='weighted'))

0.04375577696155697


In [57]:
#Adaboost
ada = AdaBoostClassifier()
ada.fit(x_train, y_train)

y_ada_pred = ada.predict(x_val)

In [58]:
print(f1_score(y_val,y_ada_pred,average='weighted'))

0.03662053266502548


In [ ]:
#KNN Classification

In [59]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train, y_train)

KNeighborsClassifier()

In [60]:
y_pred_knn = classifier.predict(x_val)

In [61]:
print(f1_score(y_val, y_pred_knn, average='weighted'))

0.1141185455773966
